<a href="https://colab.research.google.com/github/sripraks/CNN/blob/main/train_models_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib
matplotlib.use("Agg")

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [3]:
class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [4]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [5]:
# load the training and testing data, then scale it into the
# range [0, 1]
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

170500096/170498071 [==============================] - 4s 0us/step


In [6]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer","dog", "frog", "horse", "ship", "truck"]

In [8]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

In [10]:
# loop over the number of models to train
for i in np.arange(0, 5):
	# initialize the optimizer and model
	print("[INFO] training model {}/{}".format(i + 1,5))
	opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9,nesterov=True)
	model = MiniVGGNet.build(width=32, height=32, depth=3,classes=10)
	model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

	# train the network
	H = model.fit_generator(aug.flow(trainX, trainY, batch_size=64),validation_data=(testX, testY), epochs=40,
		steps_per_epoch=len(trainX) // 64, verbose=1)

	# save the model to disk
	p = ["/content/Models", "model_{}.model".format(i)]
	model.save(os.path.sep.join(p))

	# evaluate the network
	predictions = model.predict(testX, batch_size=64)
	report = classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=labelNames)

	# save the classification report to file
	p = ["/content/Models", "model_{}.txt".format(i)]
	f = open(os.path.sep.join(p), "w")
	f.write(report)
	f.close()

	# plot the training loss and accuracy
	p = ["/content/Output", "model_{}.png".format(i)]
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, 40), H.history["loss"],
		label="train_loss")
	plt.plot(np.arange(0, 40), H.history["val_loss"],
		label="val_loss")
	plt.plot(np.arange(0, 40), H.history["accuracy"],
		label="train_acc")
	plt.plot(np.arange(0, 40), H.history["val_accuracy"],
		label="val_acc")
	plt.title("Training Loss and Accuracy for model {}".format(i))
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend()
	plt.savefig(os.path.sep.join(p))
	plt.close()

[INFO] training model 1/5
Epoch 1/40
781/781 [==============================] - 32s 41ms/step - loss: 1.7849 - accuracy: 0.4007 - val_loss: 1.2772 - val_accuracy: 0.5529
Epoch 2/40
781/781 [==============================] - 34s 43ms/step - loss: 1.3544 - accuracy: 0.5228 - val_loss: 1.1956 - val_accuracy: 0.5949
Epoch 3/40
781/781 [==============================] - 32s 41ms/step - loss: 1.1746 - accuracy: 0.5873 - val_loss: 1.0141 - val_accuracy: 0.6576
Epoch 4/40
781/781 [==============================] - 32s 40ms/step - loss: 1.0660 - accuracy: 0.6248 - val_loss: 0.9132 - val_accuracy: 0.6826
Epoch 5/40
781/781 [==============================] - 32s 41ms/step - loss: 0.9960 - accuracy: 0.6491 - val_loss: 0.8881 - val_accuracy: 0.6968
Epoch 6/40
781/781 [==============================] - 31s 40ms/step - loss: 0.9331 - accuracy: 0.6717 - val_loss: 0.8463 - val_accuracy: 0.7109
Epoch 7/40
781/781 [==============================] - 31s 40ms/step - loss: 0.8988 - accuracy: 0.6828 - val_lo

In [11]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
import numpy as np
import argparse
import glob
import os

In [12]:
# load the testing data, then scale it into the range [0, 1]
(testX, testY) = cifar10.load_data()[1]
testX = testX.astype("float") / 255.0

In [15]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
	"dog", "frog", "horse", "ship", "truck"]

In [16]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
testY = lb.fit_transform(testY)

In [20]:
# construct the path used to collect the models then initialize the
# models list
modelPaths = os.path.sep.join(["/content/Models", "*.model"])
modelPaths = list(glob.glob(modelPaths))
models = []

In [21]:
# loop over the model paths, loading the model, and adding it to
# the list of models
for (i, modelPath) in enumerate(modelPaths):
	print("[INFO] loading model {}/{}".format(i + 1,
		len(modelPaths)))
	models.append(load_model(modelPath))

[INFO] loading model 1/5
[INFO] loading model 2/5
[INFO] loading model 3/5
[INFO] loading model 4/5
[INFO] loading model 5/5


In [22]:
# initialize the list of predictions
print("[INFO] evaluating ensemble...")
predictions = []

[INFO] evaluating ensemble...


In [23]:
# loop over the models
for model in models:
	# use the current model to make predictions on the testing data,
	# then store these predictions in the aggregate predictions list
	predictions.append(model.predict(testX, batch_size=64))

In [24]:
# average the probabilities across all model predictions, then show
# a classification report
predictions = np.average(predictions, axis=0)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))

              precision    recall  f1-score   support

    airplane       0.87      0.85      0.86      1000
  automobile       0.90      0.95      0.93      1000
        bird       0.82      0.68      0.74      1000
         cat       0.77      0.58      0.66      1000
        deer       0.76      0.79      0.77      1000
         dog       0.85      0.63      0.72      1000
        frog       0.62      0.97      0.75      1000
       horse       0.88      0.88      0.88      1000
        ship       0.92      0.91      0.91      1000
       truck       0.88      0.93      0.90      1000

    accuracy                           0.81     10000
   macro avg       0.83      0.81      0.81     10000
weighted avg       0.83      0.81      0.81     10000



# New Section